In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from collections import Counter
from typing import List, Dict, Tuple

# NLP libraries
import pymorphy2
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')


/Users/a.danyarov/Library/Caches/pypoetry/virtualenvs/lamoda-bootcamp-9UBAvNEw-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from sentence_transformers import SentenceTransformer
from pathlib import Path

MODEL_PATH = Path.home() / ".cache/huggingface/hub/models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2/snapshots/86741b4e3f5cb7765a600d3a3d55a0f6a6cb443d"

sentence_model = SentenceTransformer(
    str(MODEL_PATH),
    local_files_only=True
)

print("Model loaded from local snapshot ✅")


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /Users/a.danyarov/.cache/huggingface/hub/models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2/snapshots/86741b4e3f5cb7765a600d3a3d55a0f6a6cb443d.

In [2]:
PROJECT_DIR = Path().resolve()
INTERIM_DIR = PROJECT_DIR / "data/interim"
FILE_PATH = PROJECT_DIR / "data/raw/lamoda_reviews.csv"

# Try to initialize Russian morphology analyzer
try:
    morph = pymorphy2.MorphAnalyzer()
    USE_MORPH = True
except Exception as e:
    print(f"Warning: pymorphy2 initialization failed: {e}")
    print("Will use simplified text processing")
    USE_MORPH = False
    morph = None

# Initialize sentence transformer for Russian
# Увеличиваем таймауты для загрузки
import os
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '300'  # 5 минут

print("Loading sentence transformer model...")
print("(This may take several minutes if downloading for the first time)")
try:
    sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    USE_SENTENCE_TRANSFORMER = True
    print("Model loaded successfully!")
except Exception as e:
    # print(f"✗ Failed to load model: {type(e).__name__}")
    # print("\n" + "="*80)
    # print("SOLUTION: Download model manually using one of these methods:")
    # print("="*80)
    # print("\nMethod 1: Run download script in terminal:")
    # print("  python download_model.py")
    # print("\nMethod 2: Use huggingface-cli:")
    # print("  huggingface-cli download sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    # print("\nMethod 3: Manual download from browser:")
    # print("  https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    # print("  Then place files in: ~/.cache/huggingface/hub/")
    # print("\nMethod 4: If huggingface.co is blocked, use VPN or proxy")
    # print("\nFor now, continuing without sentence transformer...")
    USE_SENTENCE_TRANSFORMER = False
    sentence_model = None


Will use simplified text processing
Loading sentence transformer model...
(This may take several minutes if downloading for the first time)


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/resolve/main/modules.json (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x11e403550>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 9375618b-19ac-4c0d-b96a-bb37b80fa0bc)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/resolve/main/modules.json (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x11e401d80>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 0fef0f71-4e46-49a0-898c-ce75d681026d)')' thrown while

KeyboardInterrupt: 

In [ ]:
# Load data
print("Loading data...")
df = pd.read_csv(FILE_PATH)
print(f"Loaded {len(df)} reviews")
print(f"Unique products: {df['product_sku'].nunique()}")
print(f"Categories: {df['good_type'].unique()}")
df.head()

,comment_id,product_sku,comment_text,name,good_type,good_subtype
0,395865747,MP002XB078CD,"Купили сыну на день рождения!Теплая, но не жар...",Куртка утепленная,Clothes,OUTWEAR
1,436891792,MP002XB07Z8I,"Приятная вещь, симпатично смотрится",Свитшот,Clothes,SWEATSHIRTS
2,383386833,MP002XC00LSY,Произведено в Турции. Качество хорошее и разме...,Поло,Clothes,TEE-SHIRTS & POLOS
3,400670943,MP002XC01NLE,Классный комплект,Боди и ползунки,Clothes,TEE-SHIRTS & POLOS
4,388822372,MP002XG03J2N,"Отличные перчатки, плотные, яркие. Ребенку 8 л...",Перчатки,Accs,GLOVES & MITTENS


In [ ]:
# For faster development, use subset of data
# Uncomment to use full dataset
USE_SUBSET = True
if USE_SUBSET:
    df = df.head(50000)  # Use 50k reviews for development
    print(f"Using subset: {len(df)} reviews")

In [ ]:
# ============================================================================
# TEXT PREPROCESSING FUNCTIONS
# ============================================================================

def clean_text(text: str) -> str:
    """Очистка текста от лишних символов"""
    if pd.isna(text):
        return ""
    text = str(text)
    # Удаляем множественные пробелы
    text = re.sub(r'\s+', ' ', text)
    # Удаляем спецсимволы, оставляем буквы, цифры, пробелы и основные знаки препинания
    text = re.sub(r'[^\w\s.,!?;:()\-]', '', text)
    return text.strip()


def lemmatize_word(word: str) -> str:
    """Лемматизация одного слова"""
    if not USE_MORPH or morph is None:
        return word.lower()
    try:
        parsed = morph.parse(word)[0]
        return parsed.normal_form
    except:
        return word.lower()


def preprocess_text(text: str, lemmatize: bool = True) -> str:
    """Предобработка текста: очистка и лемматизация"""
    text = clean_text(text)
    if not text:
        return ""
    
    if lemmatize:
        words = text.split()
        lemmatized = [lemmatize_word(word) for word in words]
        return " ".join(lemmatized)
    return text.lower()


def extract_sentences(text: str) -> List[str]:
    """Разбиение текста на предложения"""
    # Простое разбиение по знакам препинания
    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 10]
    return sentences


In [ ]:
# ============================================================================
# KEY PHRASE EXTRACTION
# ============================================================================

# Словарь важных свойств товаров (можно расширить)
PRODUCT_ATTRIBUTES = {
    'качество': ['качество', 'качественный', 'качественная', 'качественные'],
    'размер': ['размер', 'размеру', 'размера', 'размеры', 'подошёл', 'подошла', 'подошли'],
    'материал': ['материал', 'ткань', 'хлопок', 'синтетика', 'кожа', 'шерсть'],
    'цвет': ['цвет', 'цвета', 'окраска', 'яркий', 'яркая', 'яркие'],
    'комфорт': ['удобный', 'удобная', 'удобные', 'комфортный', 'комфортная', 'комфортно'],
    'цена': ['цена', 'стоимость', 'дешево', 'дорого', 'дорогой', 'дешевый'],
    'стиль': ['стильный', 'стильная', 'стильные', 'модный', 'модная', 'модные'],
    'тепло': ['теплый', 'теплая', 'теплые', 'тепло', 'утепленный', 'утепленная'],
    'прочность': ['прочный', 'прочная', 'прочные', 'долговечный', 'долговечная', 'плотный', 'плотная'],
    'внешний вид': ['красивый', 'красивая', 'красивые', 'симпатичный', 'симпатично', 'смотрится']
}


def extract_key_phrases(texts: List[str], max_phrases: int = 20) -> List[Tuple[str, float]]:
    """Извлечение ключевых фраз из текстов с помощью TF-IDF"""
    if not texts:
        return []
    
    # Объединяем все тексты
    processed_texts = [preprocess_text(text, lemmatize=True) for text in texts]
    processed_texts = [t for t in processed_texts if t]
    
    if not processed_texts:
        return []
    
    # Используем TF-IDF для извлечения важных слов
    vectorizer = TfidfVectorizer(
        max_features=max_phrases * 2,
        ngram_range=(1, 2),  # Униграммы и биграммы
        min_df=2,  # Минимум 2 упоминания
        max_df=0.8  # Максимум в 80% документов
    )
    
    try:
        tfidf_matrix = vectorizer.fit_transform(processed_texts)
        feature_names = vectorizer.get_feature_names_out()
        
        # Суммируем TF-IDF по всем документам
        scores = np.asarray(tfidf_matrix.sum(axis=0)).flatten()
        
        # Сортируем по важности
        indices = scores.argsort()[-max_phrases:][::-1]
        key_phrases = [(feature_names[i], scores[i]) for i in indices if scores[i] > 0]
        
        return key_phrases
    except:
        return []


In [ ]:
# ============================================================================
# SEMANTIC CLUSTERING FOR TAG GENERATION
# ============================================================================

def cluster_sentences(sentences: List[str], n_clusters: int = 5) -> Dict[int, List[str]]:
    """Кластеризация предложений по смыслу"""
    if len(sentences) < n_clusters:
        return {i: [sentences[i]] for i in range(len(sentences))}
    
    # Если sentence transformer недоступен, используем TF-IDF для кластеризации
    if not USE_SENTENCE_TRANSFORMER or sentence_model is None:
        # Используем TF-IDF векторизацию для кластеризации
        vectorizer = TfidfVectorizer(max_features=100, ngram_range=(1, 2))
        try:
            tfidf_matrix = vectorizer.fit_transform(sentences)
            kmeans = KMeans(n_clusters=min(n_clusters, len(sentences)), random_state=42, n_init=10)
            clusters = kmeans.fit_predict(tfidf_matrix)
        except:
            # Если кластеризация не удалась, просто группируем по порядку
            return {i: [sentences[i]] for i in range(min(n_clusters, len(sentences)))}
    else:
        # Получаем эмбеддинги предложений
        embeddings = sentence_model.encode(sentences, show_progress_bar=False)
        # Кластеризация
        kmeans = KMeans(n_clusters=min(n_clusters, len(sentences)), random_state=42, n_init=10)
        clusters = kmeans.fit_predict(embeddings)
    
    # Группируем предложения по кластерам
    clustered = {}
    for idx, cluster_id in enumerate(clusters):
        if cluster_id not in clustered:
            clustered[cluster_id] = []
        clustered[cluster_id].append(sentences[idx])
    
    return clustered


def generate_tag_from_cluster(cluster_sentences: List[str]) -> str:
    """Генерация тега из кластера предложений"""
    if not cluster_sentences:
        return ""
    
    # Если только одно предложение, возвращаем его
    if len(cluster_sentences) == 1:
        tag = cluster_sentences[0]
    elif USE_SENTENCE_TRANSFORMER and sentence_model is not None:
        # Находим наиболее репрезентативное предложение (ближайшее к центру)
        embeddings = sentence_model.encode(cluster_sentences, show_progress_bar=False)
        center = embeddings.mean(axis=0)
        distances = np.linalg.norm(embeddings - center, axis=1)
        best_idx = distances.argmin()
        tag = cluster_sentences[best_idx]
    else:
        # Используем самое короткое и информативное предложение
        # Предпочитаем предложения средней длины (не слишком короткие, не слишком длинные)
        lengths = [len(s.split()) for s in cluster_sentences]
        # Ищем предложение с длиной ближе к медиане
        median_len = np.median(lengths)
        best_idx = min(range(len(lengths)), key=lambda i: abs(lengths[i] - median_len))
        tag = cluster_sentences[best_idx]
    
    # Укорачиваем тег, если он слишком длинный
    if len(tag) > 50:
        words = tag.split()
        tag = " ".join(words[:8])  # Первые 8 слов
        if len(tag) > 50:
            tag = tag[:47] + "..."
    
    return tag


In [ ]:
# ============================================================================
# MAIN PIPELINE: GENERATE TAGS FOR PRODUCT
# ============================================================================

def generate_product_tags(
    reviews: List[str],
    product_category: str = None,
    max_tags: int = 6,
    min_reviews: int = 3
) -> List[str]:
    """
    Генерация саммари-тегов для товара на основе отзывов
    
    Args:
        reviews: Список текстов отзывов
        product_category: Категория товара (опционально)
        max_tags: Максимальное количество тегов (3-6)
        min_reviews: Минимальное количество отзывов для генерации тегов
    
    Returns:
        Список саммари-тегов (до max_tags штук)
    """
    if len(reviews) < min_reviews:
        return []
    
    # Шаг 1: Предобработка отзывов
    processed_reviews = [preprocess_text(r, lemmatize=False) for r in reviews]
    processed_reviews = [r for r in processed_reviews if len(r.strip()) > 10]
    
    if not processed_reviews:
        return []
    
    # Шаг 2: Извлечение всех предложений из отзывов
    all_sentences = []
    for review in processed_reviews:
        sentences = extract_sentences(review)
        all_sentences.extend(sentences)
    
    if len(all_sentences) < 3:
        # Если предложений мало, используем ключевые фразы
        key_phrases = extract_key_phrases(processed_reviews, max_phrases=max_tags)
        tags = [phrase for phrase, _ in key_phrases[:max_tags]]
        return tags
    
    # Шаг 3: Кластеризация предложений по смыслу
    n_clusters = min(max_tags, len(all_sentences) // 2)
    n_clusters = max(3, n_clusters)  # Минимум 3 кластера
    
    clustered = cluster_sentences(all_sentences, n_clusters=n_clusters)
    
    # Шаг 4: Генерация тегов из каждого кластера
    tags = []
    for cluster_id, cluster_sents in clustered.items():
        if cluster_sents:
            tag = generate_tag_from_cluster(cluster_sents)
            if tag and len(tag) > 5:  # Минимальная длина тега
                tags.append(tag)
    
    # Шаг 5: Дополняем ключевыми фразами, если тегов мало
    if len(tags) < max_tags:
        key_phrases = extract_key_phrases(processed_reviews, max_phrases=10)
        for phrase, score in key_phrases:
            if phrase not in tags and len(phrase) > 3:
                tags.append(phrase)
                if len(tags) >= max_tags:
                    break
    
    # Шаг 6: Фильтрация и ранжирование тегов
    # Удаляем слишком похожие теги
    final_tags = []
    for tag in tags[:max_tags * 2]:  # Берем больше, чтобы отфильтровать
        if not final_tags:
            final_tags.append(tag)
            continue
        
        # Проверяем семантическую близость с уже добавленными
        if USE_SENTENCE_TRANSFORMER and sentence_model is not None:
            # Используем семантические эмбеддинги
            tag_embedding = sentence_model.encode([tag], show_progress_bar=False)[0]
            existing_embeddings = sentence_model.encode(final_tags, show_progress_bar=False)
            similarities = np.dot(existing_embeddings, tag_embedding) / (
                np.linalg.norm(existing_embeddings, axis=1) * np.linalg.norm(tag_embedding)
            )
            max_similarity = similarities.max()
        else:
            # Используем простое сравнение по словам (Jaccard similarity)
            tag_words = set(tag.lower().split())
            max_similarity = 0
            for existing_tag in final_tags:
                existing_words = set(existing_tag.lower().split())
                if len(tag_words) == 0 or len(existing_words) == 0:
                    continue
                jaccard = len(tag_words & existing_words) / len(tag_words | existing_words)
                max_similarity = max(max_similarity, jaccard)
        
        # Добавляем только если не слишком похож на существующие
        if max_similarity < 0.7:  # Порог схожести
            final_tags.append(tag)
        
        if len(final_tags) >= max_tags:
            break
    
    return final_tags[:max_tags]


In [ ]:
# ============================================================================
# BATCH PROCESSING: APPLY PIPELINE TO DATASET
# ============================================================================

def process_dataset(df: pd.DataFrame, sample_size: int = None) -> pd.DataFrame:
    """
    Обработка всего датасета: генерация тегов для каждого товара
    
    Args:
        df: DataFrame с отзывами
        sample_size: Количество товаров для обработки (None = все)
    
    Returns:
        DataFrame с тегами для каждого товара
    """
    # Группируем по SKU
    grouped = df.groupby('product_sku')
    
    results = []
    total_products = len(grouped)
    
    if sample_size:
        # Берем товары с наибольшим количеством отзывов
        review_counts = grouped.size().sort_values(ascending=False)
        top_skus = review_counts.head(sample_size).index.tolist()
        grouped = df[df['product_sku'].isin(top_skus)].groupby('product_sku')
        total_products = len(grouped)
    
    print(f"Processing {total_products} products...")
    
    for idx, (sku, group) in enumerate(grouped, 1):
        if idx % 100 == 0:
            print(f"Processed {idx}/{total_products} products...")
        
        reviews = group['comment_text'].dropna().tolist()
        category = group['good_type'].iloc[0] if 'good_type' in group.columns else None
        product_name = group['name'].iloc[0] if 'name' in group.columns else None
        
        # Генерируем теги
        tags = generate_product_tags(
            reviews=reviews,
            product_category=category,
            max_tags=6,
            min_reviews=3
        )
        
        results.append({
            'product_sku': sku,
            'product_name': product_name,
            'category': category,
            'num_reviews': len(reviews),
            'tags': tags,
            'tags_count': len(tags)
        })
    
    result_df = pd.DataFrame(results)
    return result_df


In [ ]:
# ============================================================================
# TEST PIPELINE ON SAMPLE PRODUCTS
# ============================================================================

# Тестируем на нескольких товарах
print("Testing pipeline on sample products...\n")

# Берем товары с наибольшим количеством отзывов
top_products = df.groupby('product_sku').size().sort_values(ascending=False).head(5)

for sku in top_products.index:
    product_reviews = df[df['product_sku'] == sku]['comment_text'].dropna().tolist()
    product_info = df[df['product_sku'] == sku].iloc[0]
    
    print(f"\n{'='*80}")
    print(f"Product SKU: {sku}")
    print(f"Product Name: {product_info.get('name', 'N/A')}")
    print(f"Category: {product_info.get('good_type', 'N/A')}")
    print(f"Number of reviews: {len(product_reviews)}")
    print(f"\nSample reviews:")
    for i, review in enumerate(product_reviews[:3], 1):
        print(f"  {i}. {review[:100]}...")
    
    # Генерируем теги
    tags = generate_product_tags(
        reviews=product_reviews,
        product_category=product_info.get('good_type'),
        max_tags=6
    )
    
    print(f"\nGenerated Tags ({len(tags)}):")
    for i, tag in enumerate(tags, 1):
        print(f"  {i}. {tag}")
    print()


In [ ]:
# ============================================================================
# PROCESS FULL DATASET (OR SAMPLE)
# ============================================================================

# Обрабатываем датасет
# Для быстрого тестирования используем sample_size
# Для полной обработки установите sample_size=None

SAMPLE_SIZE = 50  # Обработать топ-50 товаров по количеству отзывов
# SAMPLE_SIZE = None  # Раскомментируйте для обработки всех товаров

print("Starting batch processing...")
results_df = process_dataset(df, sample_size=SAMPLE_SIZE)

print(f"\nProcessed {len(results_df)} products")
print(f"Average tags per product: {results_df['tags_count'].mean():.2f}")
print(f"\nResults preview:")
results_df.head(10)


In [ ]:
# ============================================================================
# EXPORT RESULTS
# ============================================================================

# Сохраняем результаты
output_path = INTERIM_DIR / "product_tags.csv"
results_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"Results saved to: {output_path}")

# Также сохраняем в более читаемом формате (JSON)
import json
output_json_path = INTERIM_DIR / "product_tags.json"

# Конвертируем список тегов в строку для JSON
json_data = results_df.copy()
json_data['tags'] = json_data['tags'].apply(lambda x: json.dumps(x, ensure_ascii=False))

json_data.to_json(output_json_path, orient='records', force_ascii=False, indent=2)
print(f"Results also saved to: {output_json_path}")

# Показываем статистику
print("\n" + "="*80)
print("STATISTICS")
print("="*80)
print(f"Total products processed: {len(results_df)}")
print(f"Products with tags: {(results_df['tags_count'] > 0).sum()}")
print(f"Average tags per product: {results_df['tags_count'].mean():.2f}")
print(f"Min tags: {results_df['tags_count'].min()}")
print(f"Max tags: {results_df['tags_count'].max()}")
print(f"\nTags distribution:")
print(results_df['tags_count'].value_counts().sort_index())


In [ ]:
# ============================================================================
# VISUALIZATION: EXAMINE RESULTS
# ============================================================================

# Показываем примеры результатов для разных категорий
print("Examples by category:\n")

for category in results_df['category'].unique()[:3]:
    if pd.isna(category):
        continue
    cat_products = results_df[results_df['category'] == category].head(3)
    print(f"\n{category}:")
    print("-" * 80)
    for _, row in cat_products.iterrows():
        print(f"\n  SKU: {row['product_sku']}")
        print(f"  Name: {row['product_name']}")
        print(f"  Reviews: {row['num_reviews']}")
        print(f"  Tags ({row['tags_count']}):")
        for tag in row['tags']:
            print(f"    - {tag}")


(1774267, 6)